In [1]:
import socket
import select

In [3]:
def service_client(new_socket,request):
    file_name=""
    re
    print(request_lines[0])
    ret=re.match(r"[^/]+(/[^ ]*)",request_lines[0])
    if ret:
        file_name=ret.group(1)
        if file_name=="/":
            file_name="/index.html"
    try:
        f=open("./html"+file_name,"rb")
    except:
        response="HTTP/1.1 404 NOT FOUND\r\n"
        response+="\r\n"
        response+="file not found"
        new_socket.send(response.encode("utf-8"))
    else:
        html_content=f.read()
        f.close()
        response_body=html_content
        response_header="HTTP/1.1 200 OK\r\n"
        response_header+="Content-Length:%d\r\n" % len(response_body)
        response+="\r\n"
        response=response_header.encode("utf-8")+response_body
        new_socket.send(response
        new_socket.send(html_content)
    
# 发完消息后不断开    new_socket.close()

In [ ]:
def main():
    #创建套接字
    tcp_server_socket=socket.socket(socket.AF_INET,socket.SOCK_STREAM)
    #端口可变
    tcp_server_socket.setsockopt(socket.SOL_SOCKET,socket.SO_REUSEADDR)
    
    #绑定
    tcp_server_socket.bind(("",7890))
    #变为监听套接字
    tcp_server_socket.listen(128)
    tcp_server_socket.setblocking(False)
    
    #创建一个epoll对象
    epl=select.epoll()
    #h将监听套接字对应的fd注册到epoll中
    epl.register(tcp_server_socket.fileno(),select.EPOLLIN)
    
    
    client_socket_list=list()
    fd_event_dict={}
    while True:
        fd_event_list=epl.poll()  #默认会堵塞，直到OS监测到数据到来 通过事件通知方式 告诉这个程序，此时才会解堵塞
        
        #[(fd,event)],套接字对应的文件描述符，这个文件描述符到底是什么事件 例如 可以调用recv接收等
        for fd,event in fd_event_list:
            #等待新的客户端到来
            if fd==tcp_server_socket.fileno():
                new_socket,client_addr=tcp_server_socket.accept()
                epl.register(new_socket.fileno,select.EPOLLIN)
                fd_event_dict[new_socket.fileno]=new_socket
            elif event==select.EPOLLIN:
                #判断已经链接的客户端是否有数据发送过来
                recv_data=fd_event_dict[fd].recv(1024).decode("utf-8")
                if recv_data:
                    service_client(fd_event_dict[fd],recv_data)
                else:
                    fd_event_dict[fd].close()
                    epl.unregister(fd)
                    del fd_event_dict[fd]
    
    tcp_server_socket.close()

In [ ]:
if __name__=='__main__':
    main()